In [1]:
import tensorflow as tf
import keras
import keras_tuner as kt

In [2]:
# Load the dataset
(X_train_all, y_train_all), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_valid, y_valid = X_train_all[-5000:], y_train_all[-5000:]
X_train, y_train = X_train_all[:-5000], y_train_all[:-5000]


In [3]:
X_train = (X_train / 255.)
X_valid = (X_valid / 255.) 
X_test = (X_test / 255.)

In [4]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=2, max_value=10, default=2)
    n_neurons = hp.Int("n_neurons", min_value=200, max_value=800)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, 
                                sampling="log")

    lr_sched = hp.Choice("lr_scheduler", values=["exp", "poly"])
    if lr_sched == "exp": 
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            learning_rate,
            decay_steps=100000,
            decay_rate=0.96,
            staircase=True)
    else:
        end_learning_rate = 0.01
        decay_steps = 10000
        learning_rate_fn = keras.optimizers.schedules.PolynomialDecay(
            learning_rate,
            decay_steps,
            end_learning_rate,
            power=0.5)
    
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(10, activation="relu"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                    metrics=["accuracy"])
    return model

In [ ]:
# don't run

random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True,
    directory="my_logs", project_name="keras_tn", seed=42)
random_search_tuner.search(X_train, y_train, epochs=10, 
                               validation_data=(X_valid, y_valid))

In [ ]:
# don't run

top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [ ]:
# don't run

top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values # best hyperparameter values

In [ ]:
# don't run

best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0] 
best_trial.summary() # summary of best trial
best_trial.metrics.get_last_value("val_accuracy") # best hyperparams + validation acc

In [ ]:
# don't run

'''
If model performance is good, I can continue training it for a few epochs on the training set
(X_train_full, y_train_full), then evaluate it on the test set, and deploy it to production
'''
best_model.fit(X_train_full, y_train_full, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

In [5]:
from pathlib import Path 
from time import strftime 

def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

In [6]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [16, 32]),
            **kwargs,
        )

In [ ]:
# don't run

tuner = kt.Hyperband(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    hyperband_iterations=2, overwrite=True, directory="my_logs", project_name="keras_tn")

In [ ]:
# don't run

root_logdir = Path(tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10)
tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), 
                 callbacks=[early_stopping_cb, tensorboard_cb])

In [7]:
bayesian_tuner = kt.BayesianOptimization(
    MyClassificationHyperModel(), max_trials=30, alpha=1e-4, beta=2.6, objective="val_accuracy",
    overwrite=True, directory="my_logs", project_name="bay")

root_logdir2 = Path(bayesian_tuner.project_dir) / "tensorboard"
tensorboard_cb2 = tf.keras.callbacks.TensorBoard(root_logdir2)
early_stopping_cb2 = tf.keras.callbacks.EarlyStopping(patience=50)
bayesian_tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), epochs=150,
                 callbacks=[early_stopping_cb2, tensorboard_cb2])

Trial 4 Complete [00h 00m 52s]
val_accuracy: 0.10239999741315842

Best val_accuracy So Far: 0.10239999741315842
Total elapsed time: 00h 02m 53s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
3                 |10                |n_hidden
382               |797               |n_neurons
0.0011221         |0.002051          |learning_rate
exp               |exp               |lr_scheduler
adam              |sgd               |optimizer
16                |16                |batch_size

Epoch 1/150
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 2s 636us/step - accuracy: 0.1145 - loss: 13.5871 - val_accuracy: 0.0950 - val_loss: 14.5869
Epoch 2/150
2239/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - accuracy: 0.1016 - loss: 13.3527

KeyboardInterrupt: 